# Seeing how notebooks compare to just plain Markdown

So far, I can't complain.
This interface does seem a little nicer for some reason.
Perhaps I need to change the black theme of my [VSCode](https://code.visualstudio.com/) back to normal and see if that makes a difference in the "feels" department.
I'm still trying to keep to the rule of one sentence per line, but not sure if that will make a difference here since these are technically grouped together in "cells".

## Update to my newpost script

I had orginally created `newpost.py` to created the basic structure of new blog posts, but in [Markdown](https://daringfireball.net/projects/markdown/).
Since I'm going to start using [Jupyter Notebooks](https://jupyter.org/install.html) as well, I decided to spruce it up a bit and add support for them.
I want to keep my notebooks, vanilla.
Meaning that I don't want to shove the *metadata* required by [Pelican](https://blog.getpelican.com/) into them.
Since I didn't know how much code went into a blank notebook, I decided to just dump the *metadata* into an `.nbdata` file.
All that's left for me to do is to fire up Jupyter, create a new book, and name it exactly like the `.nbdata` file.
I didn't want to bother with typing it all out, so I used [Pyperclip](https://github.com/asweigart/pyperclip) to copy the *slug* name into my clipboard.
Now a simple *paste* gets the job done!

In [ ]:
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import List

import pyperclip

AUTHOR = "Martin Uribe"
ARTICLE_LOCATION = Path("content", "articles")


@dataclass
class Article:
    title: str
    category: str
    tags: List[str]
    summary: str
    markdown: bool

    def __post_init__(self):
        self.category = self.category.title()
        self.date = datetime.today().strftime("%Y-%m-%d %H:%M")
        self.slug = self.title.lower().replace(" ", "-")
        self.author = AUTHOR
        if self.markdown:
            self.location = ARTICLE_LOCATION.joinpath(f"{self.slug}.md")
        else:
            self.location = ARTICLE_LOCATION.joinpath(f"{self.slug}.nbdata")

    def create_file(self):
        header = f"title: {self.title}\n"
        header += f"date: {self.date}\n"
        header += f"category: {self.category}\n"
        header += f"tags: {self.tags}\n"
        header += f"slug: {self.slug}\n"
        header += f"author: {self.author}\n"
        header += f"summary: {self.summary}\n"

        if self.markdown:
            header += f"\n# {self.title}\n\n"
        else:
            pyperclip.copy(self.slug)

        self.location.write_text(header)
        print(f"Generated new article: {self.location}")


def get_bool(subject):
    value = input(f"{subject.capitalize():>10} [y]/n? ")
    if not value:
        return True
    return False


def get_input(subject):
    value = input(f"{subject:>10}: ")
    if not value:
        print(f"No subject provided!")
        exit

    return value.strip()


def get_tags():
    tags = []
    while True:
        tag = input("       tag: ")
        if tag:
            tags.append(tag.strip().lower())
        else:
            break

    return ", ".join(tags)


def main():
    title = get_input("title")
    category = get_input("category")
    tags = get_tags()
    summary = get_input("summary")
    markdown = get_bool("markdown")
    post = Article(title, category, tags, summary, markdown)
    post.create_file()


# if __name__ == "__main__":
#     main()


> I had to comment out the last bit of code, to prevent it from running in the notebook!

## Conclusion

I'll keep it short today. I basically just want to see how the output from this will look on the site and at the same time, show off my script :). I really don't like that marker that's being added to the end of the markdown cells, so I'll see if I can get rid of those...